In [1]:
from network import CNetwork

In [2]:
import pandas as pd
import numpy as np

In [3]:
import os

In [4]:
name_dataset = "dataset_1"
dataset_path = "datasetsv2/"
length_cut = 500
random_flag = True
dimension = 124
feature_selection = 'common_words'

In [5]:
df = pd.read_csv(dataset_path + name_dataset + ".csv")

In [6]:
df.head(5)

,label,text,book
0,Pelham Grenville,"\n\n\n\n\n\n\n\nProduced by Christine Gehring,...",Right Ho Jeeves
1,Pelham Grenville,"\n\n\n\n\n\n\n\nProduced by Suzanne L. Shell, ...",Tales Of St Austin
2,Joseph Conrad,And that last\n\nword was the single word of ...,Victory
3,Joseph Conrad,\n\n\n\nIf I have ever had these gifts in any ...,Under Western Eyes
4,Bram Stoker,\n\n\n\n Lond...,The Lady Of The Shroud


In [7]:
from utils.text_processing import get_min_len_corpus

In [8]:
print("Min Length:", get_min_len_corpus(list(df["text"])))

Min Length: 47826


In [9]:
from utils.text_processing import get_corpus, get_random_corpus

In [10]:
texts = list(df['text'])

In [11]:
corpus, segmented_corpus = get_corpus(texts, length_cut)

In [12]:
from utils.text_processing import remove_puntuaction

In [13]:
segmented_corpus = [[[remove_puntuaction(w) for w in t] for t in p] for p in segmented_corpus ]

In [71]:
segmented_corpus[34]

[['Produced',
  'by',
  'Meredith',
  'Bach',
  'Jane',
  'Hyland',
  'and',
  'the',
  'Online',
  'Distributed',
  'Proofreading',
  'Team',
  'at',
  'httpswwwpgdpnet',
  'This',
  'file',
  'was',
  'produced',
  'from',
  'images',
  'generously',
  'made',
  'available',
  'by',
  'The',
  'Internet',
  'ArchiveCanadian',
  'Libraries',
  'LIST',
  'OF',
  'ILLUSTRATIONS',
  'ROXANA',
  'Frontispiece',
  'THE',
  'BREWER',
  'AND',
  'HIS',
  'MEN',
  'Page',
  '12',
  'THE',
  'JEWELLER',
  'IS',
  'ABOUT',
  'TO',
  'LEAVE',
  'FOR',
  'VERSAILLES',
  '74',
  'THE',
  'VISIT',
  'OF',
  'THE',
  'PRINCE',
  '90',
  'THE',
  'DUTCH',
  'MERCHANT',
  'CALLS',
  'ON',
  'ROXANA',
  'Being',
  'the',
  'Person',
  'known',
  'by',
  'the',
  'name',
  'of',
  'the',
  'Lady',
  'Roxana',
  'in',
  'the',
  'Time',
  'of',
  'King',
  'Charles',
  'II',
  'Then',
  'for',
  'some',
  'years',
  'various',
  'editions',
  'of',
  'The',
  'Fortunate',
  'Mistress',
  'came',
  'out',

In [15]:
selected_corpus, words_features, word_index, index_word = get_random_corpus(segmented_corpus,tokenized=True,feature_selection = feature_selection)

In [80]:
wor = [(index_word[w],w) for w in selected_corpus[34]]

In [81]:
wor

[('Besides', '4340'),
 ('you', '59'),
 ('are', '673'),
 ('undone', '4343'),
 ('if', '194'),
 ('you', '59'),
 ('do', '234'),
 ('not', '216'),
 ('and', '248'),
 ('if', '194'),
 ('my', '140'),
 ('doing', '4291'),
 ('it', '95'),
 ('would', '4'),
 ('save', '3128'),
 ('you', '59'),
 ('from', '80'),
 ('being', '926'),
 ('undone', '4343'),
 ('as', '181'),
 ('I', '118'),
 ('said', '162'),
 ('before', '327'),
 ('he', '32'),
 ('shall', '886'),
 ('if', '194'),
 ('he', '32'),
 ('will', '183'),
 ('if', '194'),
 ('he', '32'),
 ('asks', '4350'),
 ('me', '37'),
 ('I', '118'),
 ('wont', '338'),
 ('deny', '4365'),
 ('him', '170'),
 ('not', '216'),
 ('I', '118'),
 ('hang', '2226'),
 ('me', '37'),
 ('if', '194'),
 ('I', '118'),
 ('do', '234'),
 ('says', '1815'),
 ('Amy', '4336'),
 ('Well', '365'),
 ('I', '118'),
 ('know', '311'),
 ('not', '216'),
 ('what', '465'),
 ('to', '8'),
 ('do', '234'),
 ('says', '1815'),
 ('I', '118'),
 ('to', '8'),
 ('Amy', '4336'),
 ('Do', '3535'),
 ('says', '1815'),
 ('Amy', '43

In [16]:
len(words_features)

6

In [17]:
words_features

{'the': 0, 'of': 1, 'a': 2, 'to': 3, 'in': 4, 'and': 5}

In [82]:
word_index['OF']

'3024'

In [19]:
len(selected_corpus)

78

In [20]:
labels = list(df['label'])

In [21]:
total_classes = list(set(labels))  ## or author
print("Total classes: {}".format(len(total_classes)))
number_books = (df[df['label'] == total_classes[0]]).shape[0]
print("Total entities for each class in train: {}".format(number_books))
dict_categories = {cat: index for index, cat in enumerate(total_classes)}

Total classes: 13
Total entities for each class in train: 6


In [22]:
y = [dict_categories[y] for y in labels]

In [23]:
total_classes

['Bram Stoker',
 'Pelham Grenville',
 'Thomas Hardy',
 'Arthur Conan Doyle',
 'Daniel Defoe',
 'Charles Darwin',
 'Hector Hugh',
 'Jane Austen',
 'Allan Poe',
 'Charles Dickens',
 'Joseph Conrad',
 'Mark Twain',
 'George Eliot']

In [118]:
def struc2vec(operating_system, networks):
    input_path = "struc2vec/graph/"
    output_path = 'struc2vec/emb/'
    try: 
        os.mkdir(input_path)
        os.mkdir(output_path)
    except:
        print("Existe")
    for i, netw in enumerate(networks):
        with open((input_path+str(i)+".edgelist"), "w") as f:
            v_names = netw.vs["name"]
            for edge in netw.get_edgelist():
                (u,v) = edge
                f.write(v_names[u] +" "+ v_names[v]+'\n')
    for i, netw in enumerate(networks):
        if operating_system == 'linux':
            path_command = "python struc2vec/src/main.py --dimensions "+str(dimension)+" --input "+input_path+str(i)+".edgelist --output "+output_path+str(i)+".emb"
            #path_command = 'python struc2vec/src/graph2vec.py --input-path ' + in_network + ' --output-path ' + extra_file + ' --dimensions ' + str(512)
            os.system(path_command)
            print("struc2vec", str(i))

In [119]:
def read_struc2vec(output_path, networks, words_features):
    all_network_features = [] 
    for i, netw in enumerate(networks):
        network_features = [[] for _ in range(len(words_features))]
        with open(output_path+str(i)+".emb", "r") as f:
            lines = f.readlines()
            num_token, dim = lines[0].split()
            for l in lines[1:]:
                row = l.strip().split(" ")
                id_node = row[0]
                if index_word[id_node].lower() in words_features:
                    emb = [float(v) for v in row[1:]]
                    network_features[words_features[index_word[id_node].lower()]] = emb
        network_features = np.array(network_features).flatten()
        print(str(i), "len",len(network_features))
        all_network_features.append(network_features)
    return np.array(all_network_features)

In [120]:
def get_struc2vec_features(sequences, words_features, index_word):
    nets = []
    for text in sequences:
        obj = CNetwork(text, model=None, index_word=index_word, percentages=None, path="")
        network = obj.create_network()
        nets.append(network)
    struc2vec("linux", nets)
    X = read_struc2vec('struc2vec/emb/', nets, words_features)
    return X

In [121]:
X = get_struc2vec_features(selected_corpus, words_features, index_word)

Nodes: 274 - Edges: 473
Nodes: 282 - Edges: 479
Nodes: 298 - Edges: 474
Nodes: 298 - Edges: 466
Nodes: 259 - Edges: 461
Nodes: 242 - Edges: 446
Nodes: 264 - Edges: 458
Nodes: 272 - Edges: 455
Nodes: 270 - Edges: 468
Nodes: 280 - Edges: 464
Nodes: 263 - Edges: 454
Nodes: 245 - Edges: 444
Nodes: 267 - Edges: 461
Nodes: 230 - Edges: 459
Nodes: 302 - Edges: 475
Nodes: 209 - Edges: 422
Nodes: 301 - Edges: 465
Nodes: 286 - Edges: 482
Nodes: 268 - Edges: 460
Nodes: 260 - Edges: 459
Nodes: 281 - Edges: 454
Nodes: 256 - Edges: 467
Nodes: 274 - Edges: 459
Nodes: 275 - Edges: 456
Nodes: 253 - Edges: 404
Nodes: 310 - Edges: 481
Nodes: 289 - Edges: 471
Nodes: 266 - Edges: 462
Nodes: 298 - Edges: 471
Nodes: 242 - Edges: 448
Nodes: 253 - Edges: 444
Nodes: 288 - Edges: 467
Nodes: 278 - Edges: 444
Nodes: 263 - Edges: 461
Nodes: 197 - Edges: 421
Nodes: 260 - Edges: 454
Nodes: 251 - Edges: 454
Nodes: 285 - Edges: 462
Nodes: 265 - Edges: 451
Nodes: 300 - Edges: 477
Nodes: 257 - Edges: 451
Nodes: 283 - Edg

In [122]:
print("Lenght of features:", X.shape)

Lenght of features: (78, 12)


In [123]:
len(X[0])

12

# Training with SVM

In [124]:
from classifierv2 import getClassifier, getClassMetrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.model_selection import train_test_split

In [125]:
RANDOM_SEED = 20
FRAC_TRAIN = 0.8

In [126]:
X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, test_size=(1.0 - FRAC_TRAIN), random_state=RANDOM_SEED)

In [127]:
X_train.shape

(62, 12)

In [128]:
X_val.shape

(16, 12)

In [129]:
from sklearn.svm import SVC

In [130]:
svc = SVC(kernel='linear', probability=True)
svc.fit(X_train, y_train)

predicted = svc.predict(X_val)
print(y_val)
print(predicted)
#print accuracy_score
print("Accuracy : " + str(accuracy_score(y_val, predicted)))

print("Micro f-measure " + str(f1_score(y_val, predicted, average='micro')))


[7, 9, 10, 0, 12, 11, 5, 11, 2, 1, 6, 8, 10, 3, 4, 3]
[11  0  1  8 11  1  8  1 12  8  1  0  9  0  8  0]
Accuracy : 0.0
Micro f-measure 0.0
